In [ ]:
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Load the dataset (replace 'your_dataset.csv' with the actual filename)
data_set = pd.read_csv("train.csv")

# Drop unnecessary columns if needed (adjust based on your data)
columns_to_drop = [
    "PID", "Street", "Alley", "Utilities", "Condition_2", "Roof_Matl",
    "Heating", "Pool_QC", "Misc_Feature", "Longitude", "Latitude"
]
data_set = data_set.drop(columns=columns_to_drop)

# Encode categorical variables as binary or one-hot
data_set = pd.get_dummies(data_set, drop_first=True)

# Split features (X) and target (y)
X = data_set.drop(columns=["Sale_Price"])
y = data_set["Sale_Price"]

# Check for and handle missing values
imputer = SimpleImputer(strategy="mean")
X = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert boolean columns to integers (just in case)
X_train = X_train.astype(int, errors='ignore')
X_test = X_test.astype(int, errors='ignore')

# Select only numeric columns for scaling
num_cols = X_train.select_dtypes(include=["float64", "int64"]).columns

# Ensure no missing values in numeric columns
X_train[num_cols] = X_train[num_cols].fillna(0)
X_test[num_cols] = X_test[num_cols].fillna(0)

# Ensure all numeric columns are float or int
X_train[num_cols] = X_train[num_cols].astype(float)
X_test[num_cols] = X_test[num_cols].astype(float)

# Standardize numerical columns
scaler = StandardScaler()
X_train[num_cols] = scaler.fit_transform(X_train[num_cols])
X_test[num_cols] = scaler.transform(X_test[num_cols])

# Train a Linear Regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Model Evaluation Metrics:")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Mean Absolute Error (MAE): {mae}")
print(f"R² Score: {r2}")


Model Evaluation Metrics:
Mean Squared Error (MSE): 2.2497059040459925e+31
Mean Absolute Error (MAE): 330869672377681.75
R² Score: -4.0127699908967195e+21


In [3]:
import pickle

# Assuming 'model' is your trained LinearRegression or other ML model
with open("ames_model.pkl", "wb") as file:
    pickle.dump(model, file)
